In [68]:
import yfinance as yf
import pandas as pd
import time
from datetime import datetime
import numpy as np

stocks = ['AMD', 'NVDA', 'META', 'GOOGL', 'TSLA', 'GM', 'AAPL', 'MSFT', 'NFLX', 'DIS', 'JPM', 'BAC', 'KO', 'PEP']
start = datetime(2016, 1, 1)
end = datetime(2018, 12, 30)

stock_prices         = yf.download(stocks, start , end, auto_adjust = False)
stock_prices = stock_prices.resample('W').last()
stock_prices.index   = stock_prices.index.tz_localize(None)      # change yf date format to match pdr
stock_prices         = stock_prices.filter(like='Adj Close')     # reduce to just columns with this in the name
stock_prices.columns = stock_prices.columns.get_level_values(1)  # tickers as col names, works no matter order of tics

stock_prices = stock_prices.unstack(level=0).reset_index()
stock_prices.columns = ['Ticker', 'Date', 'Adj Close']
stock_prices['Return'] = stock_prices.groupby('Ticker')['Adj Close'].pct_change()
stock_prices.set_index('Date', inplace=True) 
print(stock_prices.head())
stock_prices.to_csv("outputs/returns_weekly_large.csv", index=True)

[*********************100%***********************]  14 of 14 completed


           Ticker  Adj Close    Return
Date                                  
2016-01-10   AAPL  21.936222       NaN
2016-01-17   AAPL  21.974682  0.001753
2016-01-24   AAPL  22.945250  0.044168
2016-01-31   AAPL  22.022194 -0.040229
2016-02-07   AAPL  21.386499 -0.028866


In [38]:
pivot_data = stock_prices.pivot(columns='Ticker', values='Adj Close')
print(pivot_data)

Ticker           AAPL        AMD        BAC         DIS         GM      GOOGL  \
Date                                                                            
2016-01-10  21.936222   2.140000  12.412969   92.494095  23.942945  36.371613   
2016-01-17  21.974682   2.030000  11.808654   87.508263  23.975378  35.355469   
2016-01-24  22.945250   2.020000  11.073674   90.304070  23.740248  37.095650   
2016-01-31  22.022194   2.200000  11.547329   89.297577  24.032133  37.886368   
2016-02-07  21.386499   1.980000  10.575524   87.508263  23.140255  35.020569   
...               ...        ...        ...         ...        ...        ...   
2018-12-02  42.595020  21.299999  24.232344  111.835831  34.620541  55.218502   
2018-12-09  40.188354  19.459999  21.819447  109.278023  31.979309  52.080006   
2018-12-16  39.470390  19.900000  21.004333  109.492691  32.357269  52.335281   
2018-12-23  35.952213  16.930000  20.051929  101.705261  30.402925  49.326672   
2018-12-30  37.264076  17.82

In [39]:
pairs =[
    ('AAPL', 'MSFT'),
    ('GOOGL', 'META'),
    ('AMD', 'NVDA'),
    ('TSLA', 'GM'),
    ('KO', 'PEP'),
    ('NFLX', 'DIS'),
    ('JPM', 'BAC'),
    ('GOOGL', 'AAPL'),
    ('META', 'NFLX'),
    ('TSLA', 'NVDA')
]

spread_list = []

for pair in pairs:
    ticker_1, ticker_2 = pair
    spread = np.log(pivot_data[ticker_1]) - np.log(pivot_data[ticker_2])

    spread_mean = spread.mean()
    spread_std = spread.std()
    z_spread = (spread - spread_mean) / spread_std
    
    pair_df = z_spread.reset_index()
    pair_df['Date'] = pd.to_datetime(pair_df['Date'])  # Convert Date column to datetime
    pair_df.set_index('Date', inplace=True)
    pair_df['Ticker Pair'] = f'{ticker_1}-{ticker_2}'
    pair_df.columns = ['Spread', 'Ticker Pair']
    spread_list.append(pair_df[['Ticker Pair', 'Spread']])

spread_df = pd.concat(spread_list)

In [63]:
pivot_data = stock_prices.pivot(columns='Ticker', values='Return')
print(pivot_data.describe())

Ticker        AAPL         AMD         BAC         DIS          GM  \
count   155.000000  155.000000  155.000000  155.000000  155.000000   
mean      0.004039    0.018037    0.004080    0.001110    0.002337   
std       0.035224    0.094845    0.037764    0.025013    0.035312   
min      -0.112982   -0.254861   -0.099573   -0.071123   -0.085065   
25%      -0.015185   -0.036686   -0.017357   -0.014471   -0.020077   
50%       0.006513    0.012590    0.001937   -0.000304    0.001064   
75%       0.021597    0.062172    0.027590    0.015225    0.023751   
max       0.132516    0.477778    0.149245    0.072773    0.127937   

Ticker       GOOGL         JPM          KO        META        MSFT  \
count   155.000000  155.000000  155.000000  155.000000  155.000000   
mean      0.002762    0.004067    0.001635    0.002742    0.004975   
std       0.029769    0.029193    0.018240    0.037464    0.025803   
min      -0.095088   -0.073025   -0.077038   -0.166952   -0.089490   
25%      -0.012528 

In [41]:
pairs =[
    ('AAPL', 'MSFT'),
    ('GOOGL', 'META'),
    ('AMD', 'NVDA'),
    ('TSLA', 'GM'),
    ('KO', 'PEP'),
    ('NFLX', 'DIS'),
    ('JPM', 'BAC'),
    ('GOOGL', 'AAPL'),
    ('META', 'NFLX'),
    ('TSLA', 'NVDA')
]


spread_list = []

for pair in pairs:
    ticker_1, ticker_2 = pair
    spread = pivot_data[ticker_1] - pivot_data[ticker_2]
    pair_df = spread.reset_index()
    pair_df['Date'] = pd.to_datetime(pair_df['Date'])  # Convert Date column to datetime
    pair_df.set_index('Date', inplace=True)
    pair_df['Ticker Pair'] = f'{ticker_1}-{ticker_2}'
    pair_df.columns = ['Return', 'Ticker Pair']
    spread_list.append(pair_df[['Ticker Pair', 'Return']])

return_spread_df = pd.concat(spread_list)

print(return_spread_df)

           Ticker Pair    Return
Date                            
2016-01-10   AAPL-MSFT       NaN
2016-01-17   AAPL-MSFT  0.027360
2016-01-24   AAPL-MSFT  0.018672
2016-01-31   AAPL-MSFT -0.093776
2016-02-07   AAPL-MSFT  0.060624
...                ...       ...
2018-12-02   TSLA-NVDA -0.052578
2018-12-09   TSLA-NVDA  0.118171
2018-12-16   TSLA-NVDA  0.029480
2018-12-23   TSLA-NVDA -0.010357
2018-12-30   TSLA-NVDA  0.012605

[1560 rows x 2 columns]


In [42]:
final_df = spread_df.merge(return_spread_df, how = 'inner', left_on = ['Date', 'Ticker Pair'], right_on = ['Date', 'Ticker Pair'],validate = 'one_to_one')

In [43]:
final_df.to_csv("outputs/spreads_weekly_large.csv", index=True)

In [71]:
stocks = ['AMD', 'NVDA', 'META', 'GOOGL', 'TSLA', 'GM', 'AAPL', 'MSFT', 'NFLX', 'DIS', 'JPM', 'BAC', 'KO', 'PEP']
start = datetime(2019, 1, 1)
end = datetime(2019, 12, 30)

stock_prices         = yf.download(stocks, start , end, auto_adjust = False)
stock_prices = stock_prices.resample('W').last()
stock_prices.index   = stock_prices.index.tz_localize(None)      # change yf date format to match pdr
stock_prices         = stock_prices.filter(like='Adj Close')     # reduce to just columns with this in the name
stock_prices.columns = stock_prices.columns.get_level_values(1)  # tickers as col names, works no matter order of tics

stock_prices = stock_prices.unstack(level=0).reset_index()
stock_prices.columns = ['Ticker', 'Date', 'Adj Close']
stock_prices['Return'] = stock_prices.groupby('Ticker')['Adj Close'].pct_change()
stock_prices.set_index('Date', inplace=True) 
print(stock_prices.describe())


[                       0%                       ]

[*********************100%***********************]  14 of 14 completed

        Adj Close      Return
count  728.000000  714.000000
mean    88.050744    0.008063
std     83.914285    0.039631
min      3.366233   -0.155794
25%     30.179365   -0.012664
50%     55.857946    0.007765
75%    117.964020    0.027176
max    385.029999    0.277019


In [72]:

pivot_data = stock_prices.pivot(columns='Ticker', values='Adj Close')

pairs =[
    ('AAPL', 'MSFT'),
    ('GOOGL', 'META'),
    ('AMD', 'NVDA'),
    ('TSLA', 'GM'),
    ('KO', 'PEP'),
    ('NFLX', 'DIS'),
    ('JPM', 'BAC'),
    ('GOOGL', 'AAPL'),
    ('META', 'NFLX'),
    ('TSLA', 'NVDA')
]

spread_list = []

for pair in pairs:
    ticker_1, ticker_2 = pair
    spread = np.log(pivot_data[ticker_1]) - np.log(pivot_data[ticker_2])

    spread_mean = spread.mean()
    spread_std = spread.std()
    z_spread = (spread - spread_mean) / spread_std
    
    pair_df = z_spread.reset_index()
    pair_df['Date'] = pd.to_datetime(pair_df['Date'])  # Convert Date column to datetime
    pair_df.set_index('Date', inplace=True)
    pair_df['Ticker Pair'] = f'{ticker_1}-{ticker_2}'
    pair_df.columns = ['Spread', 'Ticker Pair']
    spread_list.append(pair_df[['Ticker Pair', 'Spread']])

spread_df = pd.concat(spread_list)

pivot_data = stock_prices.pivot(columns='Ticker', values='Return')

spread_list = []

for pair in pairs:
    ticker_1, ticker_2 = pair
    spread = pivot_data[ticker_1] - pivot_data[ticker_2]
    pair_df = spread.reset_index()
    pair_df['Date'] = pd.to_datetime(pair_df['Date'])  
    pair_df.set_index('Date', inplace=True)
    pair_df['Ticker Pair'] = f'{ticker_1}-{ticker_2}'
    pair_df.columns = ['Return', 'Ticker Pair']
    spread_list.append(pair_df[['Ticker Pair', 'Return']])

return_spread_df = pd.concat(spread_list)


test_df = spread_df.merge(return_spread_df, how = 'inner', left_on = ['Date', 'Ticker Pair'], right_on = ['Date', 'Ticker Pair'],validate = 'one_to_one')
test_df.to_csv("outputs/spreads_testing.csv", index=True)

print(test_df['Return'].describe())
print(pivot_data.describe())

count    510.000000
mean       0.000142
std        0.048090
min       -0.184010
25%       -0.021294
50%        0.000695
75%        0.019722
max        0.261260
Name: Return, dtype: float64
Ticker       AAPL        AMD        BAC        DIS         GM      GOOGL  \
count   51.000000  51.000000  51.000000  51.000000  51.000000  51.000000   
mean     0.013907   0.019417   0.007434   0.006287   0.003237   0.004853   
std      0.028009   0.062235   0.036013   0.030506   0.035514   0.027414   
min     -0.065222  -0.134627  -0.078157  -0.058542  -0.067076  -0.068787   
25%     -0.003909  -0.024496  -0.009016  -0.010861  -0.022816  -0.012134   
50%      0.016401   0.024667   0.009556   0.001890   0.003095   0.005585   
75%      0.029924   0.055068   0.022778   0.024324   0.019369   0.021177   
max      0.086137   0.182415   0.125624   0.130957   0.115511   0.100455   

Ticker        JPM         KO       META       MSFT       NFLX       NVDA  \
count   51.000000  51.000000  51.000000  51.000000